<a href="https://colab.research.google.com/github/pablo-arantes/EGB2025-MC14/blob/main/EGB_Virtual_Screening_Aula4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hi there!**

This is a Jupyter notebook for running a virtual screening workflow using the Gnina docking software, which utilizes an ensemble of convolutional neural networks as a scoring function.

The main goal of this notebook is to demonstrate how to harness the power of cloud-computing to perform drug binding structure prediction in a cheap and yet feasible fashion.

---

 **This notebook is NOT a standard protocol for molecular docking calculations!** It is just a simple docking pipeline illustrating each step of the process.

---


**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/pablo-arantes/Cloud-Bind/issues

**Acknowledgments**
- We would like to thank the [GNINA](https://github.com/gnina/gnina) team for doing an excellent job open sourcing the software.
- We would like to thank the [Roitberg](https://roitberg.chem.ufl.edu/) team for developing the fantastic [TorchANI](https://github.com/aiqm/torchani).
- We would like to thank [@ruiz_moreno_aj](https://twitter.com/ruiz_moreno_aj) for his work on [Jupyter Dock](https://github.com/AngelRuizMoreno/Jupyter_Dock)
- We would like to thank the ChemosimLab ([@ChemosimLab](https://twitter.com/ChemosimLab)) team for their incredible [ProLIF](https://prolif.readthedocs.io/en/latest/index.html#) (Protein-Ligand Interaction Fingerprints) tool.
- We would like to thank the [OpenBPMD](https://github.com/Gervasiolab/OpenBPMD) team for their open source implementation of binding pose metadynamics (BPMD).
- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.
- Finally, we would like to thank [Making it rain](https://github.com/pablo-arantes/making-it-rain) team, **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)), for their amazing work.
- A Cloud-Bind by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes))

- For related notebooks see: [Cloud-Bind](https://github.com/pablo-arantes/Cloud-Bind)

In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, have some coffee and wait. ;-)
# install dependencies
%%capture
import sys
import tarfile
import os
import subprocess
import sys
#subprocess.run("rm -rf /usr/local/conda-meta/pinned", shell=True)
subprocess.run("pip -q install py3Dmol", shell=True)
subprocess.run("pip install git+https://github.com/pablo-arantes/biopandas", shell=True)
subprocess.run("pip install bio", shell=True)
subprocess.run("pip install torch torchvision", shell=True)
subprocess.run("pip install torchani", shell=True)
subprocess.run("pip install ase", shell=True)
subprocess.run("pip install pandas", shell=True)
subprocess.run("pip install seaborn", shell=True)
subprocess.run("pip install openmmforcefields", shell=True)
subprocess.run("pip install datamol", shell=True)
subprocess.run("conda install -c conda-forge pdbfixer -y", shell=True)
subprocess.run("pip install parmed", shell=True)
subprocess.run("conda install openbabel", shell=True)
subprocess.run("pip install rdkit", shell=True)
subprocess.run("wget https://github.com/gnina/gnina/releases/download/v1.3/gnina", shell=True)
subprocess.run("chmod +x gnina", shell=True)
subprocess.run("git clone https://github.com/pablo-arantes/AEV-PLIG.git", shell=True)
subprocess.run("pip install logmd==0.1.30", shell=True)
subprocess.run("pip install MDAnalysis", shell=True)
subprocess.run("pip install posebusters --upgrade", shell=True)
subprocess.run("mamba install -c conda-forge pymol-open-source -y", shell=True)
subprocess.run("pip install qcelemental", shell=True)
subprocess.run("pip install torch-geometric", shell=True)
subprocess.run("wget https://github.com/rdk/p2rank/releases/download/2.5/p2rank_2.5.tar.gz", shell=True)
file = tarfile.open('p2rank_2.5.tar.gz')
file.extractall('/content/')
file.close()
os.remove('p2rank_2.5.tar.gz')
subprocess.run("pip install medchem", shell=True)

#load dependencies
import parmed as pmd
from biopandas.pdb import PandasPdb
import urllib.request
import numpy as np
import py3Dmol
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from matplotlib import colors
from IPython.display import set_matplotlib_formats
from rdkit import Chem
import datamol as dm
import seaborn as sns
from concurrent.futures import ProcessPoolExecutor

## Using Google Drive to store simulation data

Google Colab does not allow users to keep data on their computing nodes. However, we can use Google Drive to read, write, and store our simulations files. Therefore, we suggest to you to:

1.   Create a folder in your own Google Drive and copy the necessary input files there.
2.   Copy the path of your created directory. We will use it below.

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title **Check if you correctly allocated GPU nodes**

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jul 17 02:26:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


---
# **Loading the necessary input files**

At this point, we should have all libraries and dependencies installed.

**Important**: Make sure the PDB file points to the correct structure.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

**Please, don't use spaces in the files and folders names, i.e. MyDrive/protein_ligand and so on.**

In [ ]:
#@title **Please, provide the necessary input files below for receptor**:
#@markdown **Important:** Run the cell to prepare your receptor and select your reference residue for the construction of an optimal box size for the docking calculations.

#@markdown Choose between uploading your own PDB file (pdb_file) or using a PDB ID (Query_PDB_ID) to download the correct file. The appropriate chains can be selected as well.

from openmm.app.pdbfile import PDBFile

import requests
import warnings
warnings.filterwarnings('ignore')
import os
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB import is_aa
import pandas as pd
from pdbfixer import PDBFixer

Google_Drive_Path = '/content/' #@param {type:"string"}
workDir = Google_Drive_Path

workDir2 = os.path.join(workDir)
workDir_check = os.path.exists(workDir2)
if workDir_check == False:
  os.mkdir(workDir2)
else:
  pass

if os.path.exists(os.path.join(workDir, "name_residues.txt")):
  os.remove(os.path.join(workDir, "name_residues.txt"))
  os.remove(os.path.join(workDir,"name_residues_receptor.txt"))
else:
  pass

temp = os.path.join(workDir, "temp.pdb")
receptor = os.path.join(workDir, "receptor.pdb")
ligand = os.path.join(workDir, "ligand.sdf")

# Choose PDB source: Upload or PDB ID
PDB_Source = "PDB_ID" # @param ["PDB_ID","Uploaded_PDB"]

if PDB_Source == "Uploaded_PDB":
    pdb_file = 'pglb.pdb' #@param {type:"string"}
    outfnm = os.path.join(workDir, pdb_file)

elif PDB_Source == "PDB_ID":
    Query_PDB_ID = '1EVE' #@param {type:"string"}
    pdbfn = Query_PDB_ID + ".pdb"
    url = 'https://files.rcsb.org/download/' + pdbfn
    outfnm = os.path.join(workDir, pdbfn)
    try:
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)
      with open(outfnm, 'wb') as outfile:
          outfile.write(response.content)
      print(f"File downloaded to: {outfnm}")
      print(f"File size: {os.path.getsize(outfnm)} bytes")
    except requests.exceptions.RequestException as e:
      print(f"Error downloading PDB file: {e}")

print(outfnm)
# Read the PDB file
ppdb = PandasPdb().read_pdb(outfnm)
selected_chains = ['A'] # @param {"type":"raw"}

# Check if selected_chains is empty or not provided:
if 'selected_chains' not in locals() or not selected_chains:
    print("No chains selected, processing all chains.")
    # Filter chains for ATOM and HETATM records
    ppdb.df['ATOM'] = ppdb.df['ATOM']
else:
    ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['chain_id'].isin(selected_chains)]
    ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['chain_id'].isin(selected_chains)]

# Remove water molecules (HOH)
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']

# Save the filtered PDB file
ppdb.to_pdb(path=temp, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

# Prepare receptor (additional filtering)
ppdb = PandasPdb().read_pdb(outfnm)

# Remove water molecules (HOH)
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']

# Remove OXT atoms and hydrogen atoms
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['element_symbol'] != 'H']

# Save the filtered receptor PDB file
ppdb.to_pdb(path=receptor, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

fixer = PDBFixer(filename=receptor)
fixer.removeHeterogens()
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.4)
PDBFile.writeFile(fixer.topology, fixer.positions, open(receptor, 'w'))


path = '/content/'


def is_het(residue):
    res = residue.id[0]
    return res != " " and res != "W"

def aa(residue):
    res = residue.id[0]
    return res != "W"


class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        return residue == self.residue and aa(residue)

def extract_ligands(path):
    pdb = PDBParser().get_structure(temp, temp)
    io = PDBIO()
    io.set_structure(pdb)
    i = 1
    name_residues = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues.append(residue)
          print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues.txt"), "a",))
          i += 1

extract_ligands(path)

def extract_ligands2(path):
    pdb = PDBParser().get_structure(receptor, receptor)
    io = PDBIO()
    io.set_structure(pdb)
    i2 = 1
    name_residues2 = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues2.append(residue)
          print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues_receptor.txt"), "a",))
          i2 += 1

extract_ligands2(path)


dataset = pd.read_csv(os.path.join(workDir, 'name_residues.txt'), delimiter = " ", header=None)
df = pd.DataFrame(dataset)
df = df.iloc[:, [2]]
new = df.to_numpy()

dataset2 = pd.read_csv(os.path.join(workDir, 'name_residues_receptor.txt'), delimiter = " ", header=None)
df2 = pd.DataFrame(dataset2)
df2 = df2.iloc[:, [2]]
new2 = df2.to_numpy()

b = 1
res_number = []
for j in new2:
  res_number.append(b)
  b += 1

print("Residue" + " - "  + "Number" )
a = 1
for j in new:
  print(', '.join(j) + " - "  + str(a))
  a += 1

File downloaded to: /content/1EVE.pdb
File size: 432216 bytes
/content/1EVE.pdb
Residue - Number
ASP - 1
HIS - 2
SER - 3
GLU - 4
LEU - 5
LEU - 6
VAL - 7
ASN - 8
THR - 9
LYS - 10
SER - 11
GLY - 12
LYS - 13
VAL - 14
MET - 15
GLY - 16
THR - 17
ARG - 18
VAL - 19
PRO - 20
VAL - 21
LEU - 22
SER - 23
SER - 24
HIS - 25
ILE - 26
SER - 27
ALA - 28
PHE - 29
LEU - 30
GLY - 31
ILE - 32
PRO - 33
PHE - 34
ALA - 35
GLU - 36
PRO - 37
PRO - 38
VAL - 39
GLY - 40
ASN - 41
MET - 42
ARG - 43
PHE - 44
ARG - 45
ARG - 46
PRO - 47
GLU - 48
PRO - 49
LYS - 50
LYS - 51
PRO - 52
TRP - 53
SER - 54
GLY - 55
VAL - 56
TRP - 57
ASN - 58
ALA - 59
SER - 60
THR - 61
TYR - 62
PRO - 63
ASN - 64
ASN - 65
CYS - 66
GLN - 67
GLN - 68
TYR - 69
VAL - 70
ASP - 71
GLU - 72
GLN - 73
PHE - 74
PRO - 75
GLY - 76
PHE - 77
SER - 78
GLY - 79
SER - 80
GLU - 81
MET - 82
TRP - 83
ASN - 84
PRO - 85
ASN - 86
ARG - 87
GLU - 88
MET - 89
SER - 90
GLU - 91
ASP - 92
CYS - 93
LEU - 94
TYR - 95
LEU - 96
ASN - 97
ILE - 98
TRP - 99
VAL - 100
PRO - 101
S

In [ ]:
#@title **Predict ligand-binding pockets from your protein structure using P2Rank**:
#@markdown **P2Rank** is a stand-alone command line program that predicts ligand-binding pockets from a protein structure. It achieves high prediction success rates without relying on an external software for computation of complex features or on a database of known protein-ligand templates.
#@markdown P2Rank makes predictions by scoring and clustering points on the protein's solvent accessible surface. Ligandability score of individual points is determined by a machine learning based model trained on the dataset of known protein-ligand complexes. For more details see [here](https://github.com/rdk/p2rank).

import subprocess
import csv
import os
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-17-openjdk-amd64/bin/javac

print(f"Receptor file: {receptor}")
output_p2rank = os.path.join(workDir, "output_p2rank")
print(f"P2Rank output directory: {output_p2rank}")

p2rank = "/content/p2rank_2.5/prank predict -f " + str(receptor) + " -o " + str(output_p2rank)
print(f"P2Rank command: {p2rank}")

original_stdout = sys.stdout
with open('p2rank.sh', 'w') as f:
  sys.stdout = f
  print(p2rank)
  sys.stdout = original_stdout
subprocess.run(["chmod 700 p2rank.sh"], shell=True)
#subprocess.run(["./p2rank.sh"], shell=True,)
result = subprocess.run(["./p2rank.sh"], shell=True, capture_output=True, text=True)
print("P2Rank stdout:", result.stdout)
print("P2Rank stderr:", result.stderr)


# Check if the output file exists
output_file = os.path.join(workDir, "output_p2rank/receptor.pdb_predictions.csv")
if os.path.exists(output_file):
  with open(output_file, 'r') as file:
    csvreader = csv.reader(file)
    residue = []
    score = []
    center_x = []
    center_y = []
    center_z = []
    for row in csvreader:
      residue.append(row[9:10])
      score.append(row[2:3])
      center_x.append(row[6:7])
      center_y.append(row[7:8])
      center_z.append(row[8:9])

  for i in range(1,len(residue)):
    file = str((residue[i])[0]).split()
    score_end = str((score[i])[0]).split()
    center_x_end = str((center_x[i])[0]).split()
    center_y_end = str((center_y[i])[0]).split()
    center_z_end = str((center_z[i])[0]).split()
    print("Pocket " + str(i))
    print("Score = " + score_end[0])
    final_residues = []
    for i in range(0,len(file)):
      test = file[i]
      final_residues.append(int(test[2:]))
    print("Selected Residues = " + str(final_residues))
    print("Center x = "+ str(center_x_end[0]), "Center y = "+ str(center_y_end[0]), "Center z = "+ str(center_z_end[0]) + "\n")
else:
  print(f"Error: P2Rank output file not found at: {output_file}")
  print("Please, check if P2Rank executed successfully and generated the expected output.")

Receptor file: /content/receptor.pdb
P2Rank output directory: /content/output_p2rank
P2Rank command: /content/p2rank_2.5/prank predict -f /content/receptor.pdb -o /content/output_p2rank
P2Rank stdout: P2Rank 2.5

DIR: /content/p2rank_2.5/config/../test_data
DIR2: /content/p2rank_2.5/config/../test_data
DIR: /content/p2rank_2.5/config/../test_output
DIR2: /content/p2rank_2.5/config/../test_output
predicting pockets for proteins from dataset [receptor.pdb]
processing [receptor.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 8.510 seconds
results saved to directory [/content/output_p2rank]

Finished successfully in 0 hours 0 minutes 11.713 seconds.

P2Rank stderr: [INFO] Console - P2Rank 2.5
[INFO] Console - 
[INFO] Main - loading default config from [/content/p2rank_2.5/config/default.groovy]
[INFO] Console - DIR: /content/p2rank_2.5/config/../test_data
[INFO] Console - DIR2: /content/p2rank_2.5/config/../test_data
[INFO] Console - DIR: /content/p2rank_2.5/config/../test_outp

In [ ]:
#@title **Please, provide the pocket or residue number for the selection**:
#@markdown **Important:** The selected pocket or residues will be used as a reference for the construction of an optimal box size for the ligand during the docking. If you want to select more than one residue, please, use comma to separte the numbers (i.e. 147,150,155,160). **Please, DO NOT USE SPACES BETWEEN THEM.**


import re
import csv

if os.path.exists(os.path.join(workDir, "name_residue.txt")):
  os.remove(os.path.join(workDir, "name_residue.txt"))
else:
  pass

# Python code to convert string to list
def Convert(string):
	li = list(string.split(","))
	return li

def extract_ligands(path,residues):
    pdb = PDBParser().get_structure(temp, temp)
    io = PDBIO()
    io.set_structure(pdb)
    i = 1
    name_residues = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          if i == int(residues):
            # print(residues)
            print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residue.txt"), "a",))
            io.save(f"res_{i}_certo.pdb", ResidueSelect(chain, residue))
          i += 1

Selection = "Pocket" #@param ["Pocket", "Residues"]

number = '1' #@param {type:"string"}

if Selection == "Pocket":
  file = str((residue[int(number)])[0]).split()
  score_end = str((score[int(number)])[0]).split()
  center_x_end = str((center_x[int(number)])[0]).split()
  center_y_end = str((center_y[int(number)])[0]).split()
  center_z_end = str((center_z[int(number)])[0]).split()
  center_x_gnina = float(center_x_end[0])
  center_y_gnina = float(center_y_end[0])
  center_z_gnina = float(center_z_end[0])
  print("Pocket " + str(number))
  print("Score = " + score_end[0])
  print("Center x = "+ str(center_x_end[0]), "Center y = "+ str(center_y_end[0]), "Center z = "+ str(center_z_end[0]) + "\n")
  final_residues = []
  for i in range(0,len(file)):
    test = file[i]
    final_residues.append(int(test[2:]))
  residues_num = final_residues
else:
  residues_num = Convert(number)

filenames=[]
for k in range(0, len(residues_num)):
  extract_ligands(path, residues_num[k])
  filenames.append(f"res_{residues_num[k]}_certo.pdb")


with open('selection_merge.pdb', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

# reading each line from original
# text file
file1 = open('/content/selection_merge.pdb', 'r')
file2 = open('/content/selection_merge_end.pdb','w')

for line in file1.readlines():

    # reading all lines that begin
    # with "TextGenerator"
    x = re.findall("^END", line)

    if not x:
        file2.write(line)

# close and save the files
file1.close()
file2.close()

dataset = pd.read_csv(os.path.join(workDir, "name_residue.txt"), delimiter = " ", header=None)
df = pd.DataFrame(dataset)
df = df.iloc[:, [2]]
new = df.to_numpy()

print("Selected Residue" + " - "  + "Number" )
for j, i in zip(new, range(0, len(residues_num))):
# for j in new:
  print(', '.join(j) + " - "  + str(residues_num[i]))
res_box = '/content/selection_merge_end.pdb'

Pocket 1
Score = 45.71
Center x = 2.1970 Center y = 65.5299 Center z = 66.3476

Selected Residue - Number
GLY - 116
GLY - 117
GLY - 118
TYR - 120
SER - 121
GLY - 122
GLU - 198
SER - 199
TRP - 278
LEU - 281
SER - 285
ILE - 286
PHE - 287
ARG - 288
PHE - 289
PHE - 329
PHE - 330
TYR - 333
GLY - 334
HIS - 439
TYR - 69
ASP - 71
GLN - 73
TRP - 83
ASN - 84


In [ ]:
#@title **Receptor Visualization**:
#@markdown Now that the protein has been sanitized and the selection has been chosen, it is a good idea to visualize and check the protein (gray) and your selection (green).

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(receptor,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open(res_box,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title **Please, provide the necessary input files for the ligand**:

#@markdown Type the smiles or a filename (SMI, CSV or SDF format) of your molecule. **Ex: C=CC(=O)OC, molecules.smi, molecules.csv or molecules.sdf**

#@markdown Just remember that if you want to use a smi, a csv or a sdf file, you should first upload the file here in Colab or in your Google Drive, and then provide the path for the file.

#@markdown If you don't know the exact smiles for your molecule, please, check https://pubchem.ncbi.nlm.nih.gov/

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolTransforms
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor
from rdkit.Chem import PandasTools
from IPython.display import SVG
import ipywidgets as widgets
import rdkit
from rdkit.Chem.Draw import IPythonConsole
AllChem.SetPreferCoordGen(True)
from IPython.display import Image
from openbabel import pybel
import matplotlib.image as mpimg


import os

import py3Dmol


Type = "csv" #@param ["smiles", "csv", "sdf"]

smiles_or_filename = "compounds.csv" #@param {type:"string"}
smiles_or_filename = os.path.join(workDir, smiles_or_filename)

if Type == "smiles":
  mol_list = dm.read_smi(smiles_or_filename)
  data = dm.to_df(mol_list, smiles_column='Smiles')
  data["mol"] = data["Smiles"].apply(dm.to_mol)
  mols = data["mol"].tolist()
  mols = [dm.fix_mol(mol) for mol in mols]
  mols = [dm.sanitize_mol(mol, sanifix=True, charge_neutral=False) for mol in mols if mol is not None]
  mols = [dm.standardize_mol(mol, disconnect_metals=False, normalize=True, reionize=True) for mol in mols if mol is not None]
  data["mol"] = mols

elif Type == "csv":
  # Load the CSV file into a pandas DataFrame
  data = pd.read_csv(smiles_or_filename)

  #@markdown Column name containing the SMILES data (**.csv option only**)
  smiles_column = 'Smiles' #@param {type:"string"}

  # Drop rows with missing SMILES data if necessary
  data.dropna(subset=[smiles_column], inplace=True)

  # Convert the SMILES column to RDKit molecules
  data["mol"] = data[smiles_column].apply(dm.to_mol)

#  data["mol"] = data[smiles_column].apply(dm.to_mol)
  mols = data["mol"].tolist()
  mols = [dm.fix_mol(mol) for mol in mols]
  mols = [dm.sanitize_mol(mol, sanifix=True, charge_neutral=False) for mol in mols if mol is not None]
  mols = [dm.standardize_mol(mol, disconnect_metals=False, normalize=True, reionize=True) for mol in mols if mol is not None]
  data["mol"] = mols

elif Type == "sdf":
  mol_list = dm.read_sdf(smiles_or_filename)
  smi_file_path = os.path.join(workDir, "molecules.smi")
  smi_list = dm.to_smi(mol_list, smi_file_path)
  data = dm.to_df(mol_list, smiles_column='Smiles')
  data["mol"] = mol_list
  mols = data["mol"].tolist()
  mols = [dm.fix_mol(mol) for mol in mols]
  mols = [dm.sanitize_mol(mol, sanifix=True, charge_neutral=False) for mol in mols if mol is not None]
  mols = [dm.standardize_mol(mol, disconnect_metals=False, normalize=True, reionize=True) for mol in mols if mol is not None]
  data["mol"] = mols

data

[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not removing hydrogen atom without neighbors
[11:16:55] WARNING: not r

,Compound,Similarity,Smiles,Unnamed: 3,Unnamed: 4,Unnamed: 5,Smiles.1,mol
0,ZINC000002058277,0.765,CCCCCCCNC(=O)C1=CC=C(OC)C=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57eb300>
1,ZINC000019398296,0.608,COC1=CC=C(C=C1)C(=O)NC[C@H]1CCCNC1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fd4d0>
2,ZINC000019398294,0.598,COC1=CC=C(C=C1)C(=O)NC[C@@H]1CCCNC1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fd540>
3,ZINC000042771452,0.565,CC(=O)C1=CC=C(OCCC2CCNCC2)C=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fd620>
4,ZINC000042771460,0.521,CC(=O)C1=CC=C(OCC[C@H]2CCCNC2)C=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fd690>
...,...,...,...,...,...,...,...,...
806,ZINC000012382637,0.868,COC1=CC(CNC(C)C)=CC=C1OC(C)C,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c92a0>
807,ZINC000019884605,0.868,COC1=CC=C(CNCC2CC2)C=C1OC,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c9310>
808,ZINC000037491240,0.868,COC1=CC=CC(CNC(=O)CN)=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c9380>
809,ZINC000028966192,0.868,COC1=CC=CC(=C1)C1(N)CCCCC1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c93f0>


In [ ]:
#@title **Small molecule filtering**:

#@markdown Choose the type of filter you want to apply; **Options: Lipinski Rule of 5 (Ro5), Rule of 4 (Ro4), Astex Rule of 3 (Ro3), picking the centroids of clusters (Centroids) or No Filters applied**

Type = "Ro3" #@param ["Ro4", "Ro5", "Ro3", "Centroids", "NoFilter"]

data['ID'] = range(1, len(data) + 1)

df_descr = dm.descriptors.batch_compute_many_descriptors(mols)

if Type == "Ro4":
  filtered = pd.concat([data, df_descr], axis=1)
  filtered = filtered[filtered["mw"] >= 400]
  filtered = filtered[filtered["n_lipinski_hba"] >= 4]
  filtered = filtered[filtered["n_rings"] >= 4]
  filtered = filtered[filtered["clogp"] >= 4]
  filtered

elif Type == "Ro5":
  filtered = pd.concat([data, df_descr], axis=1)
  filtered = filtered[filtered["mw"] <= 500]
  filtered = filtered[filtered["n_lipinski_hba"] <= 10]
  filtered = filtered[filtered["n_lipinski_hbd"] <= 5]
  filtered = filtered[filtered["clogp"] <= 5]
  filtered

elif Type == "Ro3":
  filtered = pd.concat([data, df_descr], axis=1)
  filtered = filtered[filtered["mw"] <= 300]
  filtered = filtered[filtered["n_lipinski_hba"] <= 3]
  filtered = filtered[filtered["n_lipinski_hbd"] <= 3]
  filtered = filtered[filtered["clogp"] <= 3]
  filtered = filtered[filtered["n_rotatable_bonds"] <= 3]
  filtered

elif Type == "Centroids":
  #@markdown Only applicable to the **Centroids** filter
  n_centroids = "50" #@param {type:"string"}
  cutoff_value = "0.3" #@param {type:"string"}
  clusters, mol_clusters = dm.cluster_mols(mols, cutoff=float(cutoff_value))
  indices, centroids = dm.pick_centroids(mols, npick=int(n_centroids), threshold=float(cutoff_value), method="sphere", n_jobs=-1)
  print(str(n_centroids) + " centroids picked")
  del filtered

elif Type == "NoFilter":
  filtered = pd.concat([data, df_descr], axis=1)
  filtered

else:
  pass

filtered

,Compound,Similarity,Smiles,Unnamed: 3,Unnamed: 4,Unnamed: 5,Smiles.1,mol,ID,mw,...,sas,n_aliphatic_carbocycles,n_aliphatic_heterocyles,n_aliphatic_rings,n_aromatic_carbocycles,n_aromatic_heterocyles,n_aromatic_rings,n_saturated_carbocycles,n_saturated_heterocyles,n_saturated_rings
17,CHEMBL9113,1.000,CC1=CC=CC=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fdc40>,18,92.062600,...,1.000000,0,0,0,1,0,1,0,0,0
28,CHEMBL8712,1.000,NCC(=C=C)C1=CC=CC=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fe110>,29,145.089149,...,2.641042,0,0,0,1,0,1,0,0,0
45,CHEMBL4559928,1.000,NCCC1=CC=C(C=C)C=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57fe880>,46,147.104799,...,1.896592,0,0,0,1,0,1,0,0,0
64,CHEMBL4553663,1.000,COC1(CO)CCOC1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57ff0d0>,65,132.078644,...,3.665330,0,1,1,0,0,0,0,1,1
89,CHEMBL4553621,1.000,NCC1=CC(OCC#N)=CC=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57ffbc0>,90,162.079313,...,1.981382,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,ZINC000042405129,0.872,CCOC1=CC=C2CC[C@@H](NC)C2=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c8510>,776,191.131014,...,2.472543,1,0,1,1,0,1,0,0,0
787,ZINC000096027256,0.870,CN[C@@]1(CCCCC1=O)C1=CC=CC(OC)=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c8a50>,788,233.141579,...,3.010533,1,0,1,1,0,1,1,0,1
796,ZINC000022150589,0.869,C[C@H](NC1CC1)C1=CC=C2OCOC2=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c8e40>,797,205.110279,...,2.471634,1,1,2,1,0,1,1,0,1
800,ZINC000022150584,0.869,C[C@@H](NC1CC1)C1=CC=C2OCOC2=C1,NaN,NaN,NaN,NaN,<rdkit.Chem.rdchem.Mol object at 0x7bdde57c9000>,801,205.110279,...,2.471634,1,1,2,1,0,1,1,0,1


In [ ]:
#@title **Ligand Optimization with RDKit**:

#@markdown Choose the output name for your optimized molecules file **(sdf format)**.
import warnings
warnings.filterwarnings('ignore')
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolTransforms
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor
from rdkit.Chem import SDWriter
from IPython.display import SVG
import ipywidgets as widgets
import rdkit
from rdkit.Chem.Draw import IPythonConsole
AllChem.SetPreferCoordGen(True)
from IPython.display import Image
from openbabel import pybel
import os, sys, glob


import py3Dmol

Optimized_Mol_File = "compounds_opt.sdf" #@param {type:"string"}
Optimized_Mol_File = os.path.join(workDir, Optimized_Mol_File)

# Criando o escritor SDF
writer = Chem.SDWriter(Optimized_Mol_File)

if 'filtered' in globals():
# Iterando pelas moléculas no DataFrame
  for i, (mol, mol_id) in enumerate(zip(filtered["mol"], filtered["ID"])):
      print(f"Processing molecule {i+1}/{len(filtered)}...")
      if mol is None:  # Checar se a molécula é inválida
          print(f"Invalid molecule on line {i}. Ignoring...")
          continue

    # Adicionar hidrogênios
      hmol = Chem.AddHs(mol)

    # Embedding 3D coordenadas
      status = AllChem.EmbedMolecule(hmol, maxAttempts=500, useRandomCoords=True)
      if status != 0:  # Verifica se a geração da conformação falhou
         print(f"Embedding failed for molecule on line {i}. Ignoring...")
         continue

    # Otimização de geometria com MMFF
      mp = AllChem.MMFFGetMoleculeProperties(hmol)
      ff = AllChem.MMFFGetMoleculeForceField(hmol, mp)
      AllChem.OptimizeMolecule(ff, maxIters=1000)

    # Salvar molécula otimizada temporariamente
      mol_file = os.path.join(workDir, f"{i}.mol")
      Chem.MolToMolFile(hmol, mol_file)

    # Recarregar a molécula otimizada para incluir no SDF
      mol2 = Chem.MolFromMolFile(mol_file, removeHs=False)
      if mol2 is not None:
        # Definir o título da molécula como "mol_X", onde X é o valor da coluna "ID"
          mol2.SetProp("_Name", f"mol_{mol_id}")
          writer.write(mol2, confId=0)
      else:
          print(f"Error saving/reading optimized molecule on line {i}.")

      # Limpar arquivos temporários
      for f in glob.glob(os.path.join(workDir, "*.mol")):
         os.remove(f)
      for f in glob.glob(os.path.join(workDir, "*.xyz")):
         os.remove(f)

# Fechar o escritor SDF
  writer.close()

elif 'centroids' in globals():# Assuming 'centroids' is a list or iterable containing the molecules
  for i, mol in enumerate(centroids):
      print(f"Processing molecule {i+1}/{len(centroids)}...")
      if mol is None:  # Check if the molecule is invalid
          print(f"Invalid molecule on line {i}. Ignoring...")
          continue

    # Add hydrogens
      hmol = Chem.AddHs(mol)

    # Embed 3D coordinates
      status = AllChem.EmbedMolecule(hmol, maxAttempts=500, useRandomCoords=True)
      if status != 0:  # Check if conformation generation failed
          print(f"Embedding failed for molecule on line {i}. Ignoring...")
          continue

    # Geometry optimization with MMFF
      mp = AllChem.MMFFGetMoleculeProperties(hmol)
      ff = AllChem.MMFFGetMoleculeForceField(hmol, mp)
      AllChem.OptimizeMolecule(ff, maxIters=1000)

    # Save optimized molecule temporarily
      mol_file = os.path.join(workDir, f"{i}.mol")
      Chem.MolToMolFile(hmol, mol_file)

    # Reload the optimized molecule to include in the SDF
      mol2 = Chem.MolFromMolFile(mol_file, removeHs=False)
      if mol2 is not None:
        # Set the molecule title as "mol_X", where X is the sequential number
          mol2.SetProp("_Name", f"mol_{i+1}")  # Use i+1 to start numbering from 1
          writer.write(mol2, confId=0)
      else:
          print(f"Error saving/reading optimized molecule on line {i}.")

    # Clean up temporary files
      for f in glob.glob(os.path.join(workDir, "*.mol")):
          os.remove(f)
      for f in glob.glob(os.path.join(workDir, "*.xyz")):
          os.remove(f)
else:
  pass
# Close the SDF writer
  writer.close()

Processing molecule 1/78...
Processing molecule 2/78...
Processing molecule 3/78...
Processing molecule 4/78...
Processing molecule 5/78...
Processing molecule 6/78...
Processing molecule 7/78...
Processing molecule 8/78...
Processing molecule 9/78...
Processing molecule 10/78...
Processing molecule 11/78...
Processing molecule 12/78...
Processing molecule 13/78...
Processing molecule 14/78...
Processing molecule 15/78...
Processing molecule 16/78...
Processing molecule 17/78...
Processing molecule 18/78...
Processing molecule 19/78...
Processing molecule 20/78...
Processing molecule 21/78...
Processing molecule 22/78...
Processing molecule 23/78...
Processing molecule 24/78...
Processing molecule 25/78...
Processing molecule 26/78...
Processing molecule 27/78...
Processing molecule 28/78...
Processing molecule 29/78...
Processing molecule 30/78...
Processing molecule 31/78...
Processing molecule 32/78...
Processing molecule 33/78...
Processing molecule 34/78...
Processing molecule 35/

In [ ]:
#@title **Parameters for the docking calculation:**

#@markdown Please choose the name of the output file from the docking calculation **(do not add file extension)**:

Output_file = "output_docking_compounds" #@param {type:"string"}
Output_file = os.path.join(workDir, Output_file)
#@markdown Amount of buffer space to add the generated box (Angstroms):

size = 10 #@param {type:"slider", min:1, max:20, step:1}

#@markdown Exhaustiveness of the global search (roughly proportional to time):
exhaustiveness = 10 #@param {type:"slider", min:2, max:64, step:2}

#@markdown Explicit random seed:
seed = "0" #@param {type:"string"}

#@markdown Convolutional neural network (CNN) parameter:

cnn_scoring = "rescore (default)" #@param ["none", "rescore (default)", "refinement", "all"]
if cnn_scoring == "rescore (default)":
  cnn_scoring = "rescore"
  scoring_vinardo = " "
elif cnn_scoring == "none":
  scoring_vinardo = " --scoring vinardo "
else:
  scoring_vinardo = " "

#@markdown **cnn_scoring** determines at what points of the docking procedure that the CNN scoring function is used.

#@markdown **none** - No CNNs used for docking. Here, uses all the empirical scoring from [Vinardo](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0155183) scoring function.

#@markdown **rescore** (default) - CNN used for reranking of final poses. Least computationally expensive CNN option.

#@markdown **refinement** - CNN used to refine poses after Monte Carlo chains and for final ranking of output poses. 10x slower than rescore when using a GPU.

#@markdown **all** - CNN used as the scoring function throughout the whole procedure. Extremely computationally intensive and not recommended.

#@markdown The default CNN scoring function is an ensemble of 5 models selected to balance pose prediction performance and runtime: dense, general_default2018_3, dense_3, crossdock_default2018, and redock_default2018.

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

docking_output_gz = os.path.join(workDir, Output_file + ".sdf.gz")
docking_output = os.path.join(workDir, Output_file + ".sdf")

if os.path.exists(docking_output_gz):
  os.remove(docking_output_gz)
elif os.path.exists(docking_output):
  os.remove(docking_output)
else:
  pass


if Selection == "Pocket":
  gnina = "./gnina -r " + str(receptor) + " -l " +  str(Optimized_Mol_File) + " --center_x " + str(center_x_gnina) +  " --center_y " + str(center_y_gnina) +  " --center_z " + str(center_z_gnina) + " --size_x " + str(size) +  " --size_y " + str(size) +  " --size_z " + str(size) + " --cnn_scoring " + str(cnn_scoring) + " --exhaustiveness " + str(exhaustiveness) + " -o " + str(docking_output_gz) + str(scoring_vinardo) +  "--num_modes 10 " + "--seed " + str(int(seed))
else:
  gnina = "./gnina -r " + str(receptor) + " -l " +  str(Optimized_Mol_File) + " --autobox_ligand " + str(res_box) +  " --autobox_add " + str(size) + " --cnn_scoring " + str(cnn_scoring) + " --exhaustiveness " + str(exhaustiveness) + " -o " + str(docking_output_gz) + str(scoring_vinardo) +  "--num_modes 10 " + "--seed " + str(int(seed))

zip_gnina = "gunzip " + str(docking_output_gz)

original_stdout = sys.stdout # Save a reference to the original standard output
with open('gnina.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(gnina)
    print(zip_gnina)
    sys.stdout = original_stdout # Reset the standard output to its original value

!chmod 700 gnina.sh 2>&1 1>/dev/null
!bash gnina.sh

import gzip

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Recommend running with single model (--cnn fast)
or without cnn scoring (--cnn_scoring=none).

Commandline: ./gnina -r /content/receptor.pdb -l /content/compounds_opt.sdf --center_x 2.197 --center_y 65.5299 --center_z 66.3476 --size_x 10 --size_y 10 --size_z 10 --cnn_scoring rescore --exhaustiveness 10 -o /content/output_docking_compounds.sdf.gz --num_modes 10 --seed 0
Using random seed: 0

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
mol_18 | pose 0 | initial pose not within box

mode |  affinity  |  intramol  |    CNN     |   

In [ ]:
import pandas as pd
from rdkit.Chem import rdFMCS,AllChem, Draw, PandasTools
import seaborn as sns
from openbabel import pybel

#@title **Docking Analysis:**

#@markdown Please choose which parameter will be used to sort the results:
Parameter = "CNN_VS" #@param ["minimizedAffinity", "CNNscore", "CNN_VS", "CNNaffinity"]

#@markdown Please fill in the blanks with the path to the sdf file from the docking output:

# Input file path
Docking_output = "output_docking_compounds.sdf" #@param {type:"string"}
Docking_output = os.path.join(workDir, Docking_output)

# Read the entire content of the file
with open(Docking_output, "r") as infile:
    lines = infile.readlines()

# Process the lines to update the titles
molecule_count = 0  # Counter for the current molecule (e.g., mol_1, mol_2)
solution_count = 0  # Counter for the solution number (e.g., _1, _2, ..., _10)
current_molecule = None  # Stores the current molecule title (e.g., mol_1)

for i, line in enumerate(lines):
    # Check if the line contains a molecule title (e.g., mol_1)
    if line.startswith("mol_"):
        # Extract the molecule number (e.g., 1 from mol_1)
        molecule_number = line.strip().split("_")[1]

        # If this is a new molecule, reset the solution counter
        if molecule_number != current_molecule:
            current_molecule = molecule_number
            solution_count = 0

        # Increment the solution counter
        solution_count += 1

        # Create the new title (e.g., mol_1_1, mol_1_2, etc.)
        new_title = f"mol_{molecule_number}_{solution_count}\n"

        # Update the line in the list
        lines[i] = new_title

# Write the updated content back to the input file
with open(Docking_output, "w") as outfile:
    outfile.writelines(lines)

# Load and process docking results
VinaPoses=PandasTools.LoadSDF(Docking_output)
AllPoses=pd.concat([VinaPoses])

# List to store scores and titles
scores = []

# Read the SDF file
for mol in pybel.readfile('sdf', Docking_output):
    molecule_title = mol.title.strip()

    # Create a dictionary to store the scores
    score_data = {
        'Molecule_Solution': molecule_title,
        'minimizedAffinity': float(mol.data['minimizedAffinity']),
        'CNNscore': float(mol.data['CNNscore']),
        'CNNaffinity': float(mol.data['CNNaffinity']),
        'CNN_VS': float(mol.data['CNN_VS']),
    }

    scores.append(score_data)

# Create a DataFrame from the list of dictionaries
scores_df = pd.DataFrame(scores)

# Reorder columns
scores_df = scores_df[['Molecule_Solution', 'minimizedAffinity', 'CNNscore', 'CNNaffinity', 'CNN_VS']]

# Sort the DataFrame based on the selected parameter
if Parameter == "minimizedAffinity":
    scores_sorted = scores_df.sort_values(by=Parameter, ascending=True).reset_index(drop=True)
else:
    scores_sorted = scores_df.sort_values(by=Parameter, ascending=False).reset_index(drop=True)

scores_sorted.to_csv(os.path.join(workDir, Parameter + "_sorted.csv"), index=False)
#scores_sorted

# New code to write the sorted molecules to a new SDF file
# Dictionary to store molecules by their titles
molecule_dict = {}

# Read the SDF file again and store molecules in the dictionary
for mol in pybel.readfile('sdf', Docking_output):
    molecule_title = mol.title.strip()
    molecule_dict[molecule_title] = mol

# Write the molecules to a new SDF file in the order specified by the sorted DataFrame
#@markdown Please choose the name of the output sdf file sorted according to the Parameter selected:
Output_sdf = "compounds_sorted_CNN_VS.sdf" #@param {type:"string"}
Output_sdf = os.path.join(workDir, Output_sdf)

with open(Output_sdf, 'w') as outfile:
    for molecule_solution in scores_sorted['Molecule_Solution']:
        if molecule_solution in molecule_dict:
            mol = molecule_dict[molecule_solution]
            outfile.write(mol.write(format='sdf'))
        else:
            print(f"Warning: Molecule '{molecule_solution}' not found in the SDF file.")

print(f"Sorted SDF file saved to {Output_sdf}")
scores_sorted

[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[13:57:45] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol

Sorted SDF file saved to /content/compoundsl_sorted_CNN_VS.sdf


,Molecule_Solution,minimizedAffinity,CNNscore,CNNaffinity,CNN_VS
0,mol_498_1,-7.98865,0.984404,6.453800,6.353143
1,mol_654_1,-7.74761,0.949792,6.463899,6.139360
2,mol_568_1,-7.26862,0.956964,6.182816,5.916734
3,mol_281_1,-6.64969,0.932533,6.179713,5.762786
4,mol_442_1,-5.86576,0.975011,5.846958,5.700850
...,...,...,...,...,...
775,mol_170_10,-4.47453,0.496912,3.209677,1.594928
776,mol_411_7,-5.24578,0.404737,3.748410,1.517121
777,mol_411_9,-6.57598,0.367817,3.992892,1.468652
778,mol_411_8,-5.46090,0.368475,3.676979,1.354875


In [ ]:
# @title **Filter for the top 10 molecules in the output sdf file**

from rdkit import Chem
import pandas as pd
import os

sdf_file = Output_sdf

def filter_top_molecules(sdf_file, num_molecules=10):
    """Filters the top N molecules from an SDF file, keeping original titles.

    Args:
        sdf_file: Path to the SDF file.
        num_molecules: Number of top molecules to extract (default is 10).

    Returns:
        A list of RDKit Mol objects representing the top molecules.
        Returns an empty list if the file does not exist or if an error occurs.
    """
    if not os.path.exists(sdf_file):
        print(f"Error: File not found - {sdf_file}")
        return []

    try:
        suppl = Chem.SDMolSupplier(sdf_file)
        top_molecules = []
        for i, mol in enumerate(suppl):
            if mol is not None:  # Check for valid molecules
                top_molecules.append(mol)
                if i + 1 == num_molecules:
                    break  # Stop after extracting the desired number

        return top_molecules

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage: Assuming 'Output_sdf' is defined from previous code
top_molecules = filter_top_molecules(Output_sdf)  # Using Output_sdf

if top_molecules:
    print(f"Successfully extracted {len(top_molecules)} molecules.")

    # Example to save to a new SDF file, keeping original titles:
    writer = Chem.SDWriter(os.path.join(workDir, "top_10_molecules.sdf"))
    for mol in top_molecules:
        writer.write(mol)
    writer.close()

Successfully extracted 10 molecules.
